# Analyse data

In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
det_log = './det/check_nyu'
track_log = './track/logs'
mode_list = ['lowerbound','disco', 'upperbound'] #'when2com',
mode_list_print = ['lowerbound','disco     ',  'upperbound'] #'when2com  ',
bootstrap_start = 0
bootstrap_end = 25
rsu = 'no_rsu'

## Detection

In [4]:
def extract_one_det_results(file):
    map_5 = []
    map_7 = []
    with open(file, 'r') as f:
        lines = f.readlines()
        for i in range(len(lines)):
            if lines[i][:7] != 'average':
                continue
            split = lines[i].split()
            map_5.append(float(split[4]))
            map_7.append(float(split[10]))
            if len(map_5) >= bootstrap_end:
                break
        f.close()
    map_result = []
    map_result.append(map_5)
    map_result.append(map_7)
    return map_result

def extract__det_results():
    result = []
    for i in range(len(mode_list)):
        file_path = os.path.join(mode_list[i], rsu)
        file_path = os.path.join(file_path, 'log_test.txt')
        file_path = os.path.join(det_log, file_path)
        print(file_path)
        temp_result = extract_one_det_results(file_path)
        result.append(temp_result)
    return result

def compute_mean_variance(det_results):
    result = []
    for i in len(det_results):
        for j in len(det_results[i]):
            temp = []
            mean = np.mean(det_results[i][j])
            std = np.std()

In [5]:
det_results = extract__det_results()
mean = np.mean(det_results, axis = 2)
std = np.std(det_results, axis = 2)

./det/check_nyu/lowerbound/no_rsu/log_test.txt
./det/check_nyu/disco/no_rsu/log_test.txt
./det/check_nyu/upperbound/no_rsu/log_test.txt


In [6]:
print("mode          mAP(0.5)          mAP(0.7)")
for i in range(len(mean)):
    st = mode_list_print[i] + "    {0:.3f}".format(mean[i][0]) + "({0:.5f})".format(std[i][0])+ "    {0:.3f}".format(mean[i][1]) + "({0:.5f})".format(std[i][1])
    print(st)

mode          mAP(0.5)          mAP(0.7)
lowerbound    0.497(0.00505)    0.441(0.00303)
disco         0.698(0.00646)    0.643(0.00495)
upperbound    0.707(0.00347)    0.670(0.00246)


In [6]:
for i in range(len(mean)):
    print(mode_list[i], " mAP for 0.5:   ", det_results[i][0])
    print(mode_list[i], " mAP for 0.7:   ", det_results[i][1])

lowerbound  mAP for 0.5:    [0.49900713562965393, 0.4939224421977997, 0.4988515377044678, 0.49761611223220825, 0.49413973093032837, 0.4971027970314026, 0.4889352321624756, 0.4948710799217224, 0.48600688576698303, 0.501207709312439, 0.49157965183258057, 0.49067413806915283, 0.5018049478530884, 0.4974174201488495, 0.5072156190872192, 0.4910263419151306, 0.5050339102745056, 0.5023539066314697, 0.49121975898742676, 0.49950000643730164, 0.4985678493976593, 0.49659666419029236, 0.49934840202331543, 0.49260687828063965, 0.500501275062561]
lowerbound  mAP for 0.7:    [0.4421604871749878, 0.44027259945869446, 0.4383939504623413, 0.44108614325523376, 0.4399482011795044, 0.4382987916469574, 0.4359024167060852, 0.4399508833885193, 0.4344748258590698, 0.4429028630256653, 0.4399273693561554, 0.4362139403820038, 0.44486764073371887, 0.44215914607048035, 0.4484386146068573, 0.43725186586380005, 0.4426913857460022, 0.44167232513427734, 0.4382147192955017, 0.44344666600227356, 0.4422401189804077, 0.4389

### Analyse each scene

In [10]:
def show_scene_mean_std():
    result = []
    #mode_list = ["upperbound"]
    for i in range(len(mode_list)):
        file_path = os.path.join(mode_list[i], rsu)
        file_path = os.path.join(file_path, 'all_scenes_mean_ap.npy')
        file_path = os.path.join(det_log, file_path)
        print(mode_list[i])
        print(file_path)
        data = np.load(file_path, allow_pickle=True).item()
        for key, value in data.items():
            mean = np.mean(value, axis=0)
            std = np.std(value, axis=0)
            string = "{}: ".format(key) + "    {0:.3f}".format(mean[0][-1]) + "({0:.5f})".format(std[0][-1])+ "    {0:.3f}".format(mean[1][-1]) + "({0:.5f})".format(std[1][-1])
            print(string)
    return

show_scene_mean_std()

lowerbound
./det/check_nyu/lowerbound/no_rsu/all_scenes_mean_ap.npy
27:     0.385(0.00677)    0.286(0.00810)
92:     0.693(0.00479)    0.636(0.00526)
5:     0.604(0.01024)    0.513(0.00859)
28:     0.444(0.00504)    0.387(0.00437)
97:     0.414(0.00555)    0.385(0.00541)
29:     0.586(0.01067)    0.455(0.00509)
8:     0.575(0.00392)    0.514(0.00374)
96:     0.426(0.00484)    0.408(0.00445)
91:     0.573(0.00470)    0.550(0.00430)
19:     0.504(0.01226)    0.469(0.00779)
disco
./det/check_nyu/disco/no_rsu/all_scenes_mean_ap.npy
91:     0.676(0.01057)    0.656(0.00858)
97:     0.703(0.01141)    0.678(0.00880)
28:     0.651(0.00794)    0.580(0.00705)
19:     0.757(0.01711)    0.654(0.01411)
8:     0.782(0.00310)    0.768(0.00485)
5:     0.795(0.00339)    0.773(0.00330)
27:     0.476(0.00640)    0.390(0.00754)
29:     0.715(0.00746)    0.655(0.00858)
96:     0.707(0.01065)    0.625(0.00906)
92:     0.723(0.01173)    0.672(0.00951)
upperbound
./det/check_nyu/upperbound/no_rsu/all_scenes_me

In [17]:
print(data)

{'8': [[0.7726448178291321, 0.8610109686851501, 0.7444735169410706, 0.6317248344421387, 0.7540466785430908, 0.7678820490837097], [0.7553267478942871, 0.8504055142402649, 0.7431247234344482, 0.6291897892951965, 0.7302101254463196, 0.7565855383872986]], '97': [[0.5423259735107422, 0.6667993068695068, 0.8496435284614563, 0.8849015831947327, 0.6952884197235107, 0.7178959846496582], [0.50236976146698, 0.6446855068206787, 0.8391231298446655, 0.8628597259521484, 0.6907229423522949, 0.6983575820922852]], '19': [[0.6668508648872375, 0.9539629817008972, 0.8230437636375427, 0.780773401260376, 0, 0.8017301559448242], [0.6447429060935974, 0.9262373447418213, 0.80611652135849, 0.7189436554908752, 0, 0.7688127756118774]], '28': [[0.5066487789154053, 0.6680319309234619, 0.566403329372406, 0, 0, 0.5796363949775696], [0.48355937004089355, 0.6347523331642151, 0.44632434844970703, 0, 0, 0.5214268565177917]], '27': [[0.29866862297058105, 0.4984737038612366, 0.4558710753917694, 0, 0, 0.41345080733299255], [

## Tracking

In [8]:
metric_list = ['bootstrap', 'MOTA', 'MOTP', 'HOTA', 'DetA', 'AssA', 'DetRe', 'DetPr', 'AssRe', 'AssPr', 'LocA']
def extract_track_result():
    result = []
    for i in range(len(mode_list)):
        file_path = os.path.join(track_log, mode_list[i])
        file_path = os.path.join(file_path, "logs_no_rsu_metric_all.npy")
        print(file_path)
        data = np.load(file_path, allow_pickle=True)
        result.append(data.item()['mean'][bootstrap_start:bootstrap_end])
    return result

In [10]:
track_results = extract_track_result()

./track/logs/lowerbound/logs_no_rsu_metric_all.npy
./track/logs/disco/logs_no_rsu_metric_all.npy
./track/logs/when2com/logs_no_rsu_metric_all.npy
./track/logs/upperbound/logs_no_rsu_metric_all.npy


In [11]:
track_mean = np.mean(track_results, axis = 1)
track_std = np.std(track_results, axis = 1)

In [12]:
def print_metric(start, end):
    title = "mode"
    for i in range(start, end):
        title += "           " + metric_list[i]
    print(title)
    for i in range(len(mean)):
        st = mode_list_print[i]
        for j in range(start, end):
            st += "  {0:.3f}".format(track_mean[i][j]) + "({0:.3f})".format(track_std[i][j])
        print(st)

In [13]:
print_metric(1,6)
print_metric(6,len(metric_list))

mode           MOTA           MOTP           HOTA           DetA           AssA
lowerbound  35.515(0.504)  84.125(0.268)  34.279(0.246)  33.548(0.348)  36.342(0.246)
disco       56.090(0.662)  86.182(0.160)  44.471(0.277)  51.802(0.499)  39.204(0.188)
when2com    29.203(0.849)  85.682(0.382)  30.750(0.458)  27.519(0.840)  35.391(0.263)
upperbound  58.119(0.414)  85.462(0.151)  44.789(0.178)  53.029(0.296)  38.855(0.158)
mode           DetRe           DetPr           AssRe           AssPr           LocA
lowerbound  34.984(0.391)  82.410(0.370)  46.895(0.316)  58.683(0.208)  86.342(0.177)
disco       54.189(0.540)  86.300(0.569)  50.795(0.224)  59.061(0.121)  88.033(0.119)
when2com    28.304(0.912)  85.662(0.613)  46.410(0.264)  59.271(0.387)  87.748(0.301)
upperbound  55.180(0.281)  86.331(0.275)  50.221(0.187)  58.614(0.150)  87.329(0.114)


In [3]:
dic= {"1":1, "2":2}
print(list(dic.keys()))

['1', '2']
